<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B35%25Acc%5D%20Hybrid%20LR-Bi-LSTM%20with%20hybrid%20adversarial%20attack%20FGSM-PGD%20based%20on%20UNSW-NB15%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hybrid LR-Bi-LSTM with hybrid adversarial attack FGSM-PGD based on UNSW-NB15 dataset**

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import np_utils
from imblearn.over_sampling import SMOTE

from keras.optimizers import Adam
#from keras.utils import np_utils
from keras.regularizers import l2
import matplotlib.pyplot as plt
#from imblearn.over_sampling import SMOTE

# Adversarial Training Imports
from keras import backend as K
import tensorflow as tf

In [2]:
# Load dataset
dataset = pd.read_csv("Dataset_10Classes.csv")
dataset = dataset.dropna() # Remove missing values
#X = dataset.drop(['Class'], axis=1)
#y = dataset['Class']

In [3]:
dataset.isnull().sum()

,0
ID_Cb,0
id,0
dur,0
proto,0
service,0
state,0
spkts,0
dpkts,0
sbytes,0
dbytes,0


In [5]:
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257673 entries, 0 to 257672
Data columns (total 47 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ID_Cb              257673 non-null  int64  
 1   id                 257673 non-null  int64  
 2   dur                257673 non-null  float64
 3   proto              257673 non-null  object 
 4   service            257673 non-null  object 
 5   state              257673 non-null  object 
 6   spkts              257673 non-null  int64  
 7   dpkts              257673 non-null  int64  
 8   sbytes             257673 non-null  int64  
 9   dbytes             257673 non-null  int64  
 10  rate               257673 non-null  float64
 11  sttl               257673 non-null  int64  
 12  dttl               257673 non-null  int64  
 13  sload              257673 non-null  float64
 14  dload              257673 non-null  float64
 15  sloss              257673 non-null  int64  
 16  dl

In [6]:
print(dataset["Class"].unique())  # Unique target classes

[0 2 1 5 8 7 4 3 9 6]


In [7]:
import collections as c
# Encode non-numeric features
categorical_columns = dataset.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    dataset[col] = le.fit_transform(dataset[col])
    label_encoders[col] = le  # Save the encoder for later decoding if needed

# Split into features and target
X = dataset.drop(['Class'], axis=1).values
y = dataset['Class'].values

# Display dataset shape and data types
print("Features (X):\n", X)
print("Target (y):\n", y)

# Check class distribution before balancing
counter = c.Counter(y)
print("Before SMOTE:", counter)

Features (X):
 [[1.000000e+00 1.000000e+00 1.214780e-01 ... 0.000000e+00 6.000000e+00
  0.000000e+00]
 [2.000000e+00 2.000000e+00 6.499020e-01 ... 0.000000e+00 6.000000e+00
  0.000000e+00]
 [3.000000e+00 3.000000e+00 1.623129e+00 ... 0.000000e+00 6.000000e+00
  0.000000e+00]
 ...
 [2.576710e+05 8.233000e+04 0.000000e+00 ... 1.000000e+00 6.000000e+00
  0.000000e+00]
 [2.576720e+05 8.233100e+04 0.000000e+00 ... 1.000000e+00 6.000000e+00
  0.000000e+00]
 [2.576730e+05 8.233200e+04 9.000000e-06 ... 0.000000e+00 6.000000e+00
  0.000000e+00]]
Target (y):
 [0 0 0 ... 0 0 0]
Before SMOTE: Counter({0: 93000, 6: 58871, 4: 44525, 5: 24246, 3: 16353, 7: 13987, 1: 2677, 2: 2329, 8: 1511, 9: 174})


In [8]:
##Installing imblearn
#!pip install -U imbalanced-learn
#!pip install imbalanced-learn

In [ ]:
## check version number
#import imblearn
#print(imblearn.__version__)

In [ ]:
## Data preprocessing
#y = dataset['Class'].values
#X = dataset.drop(['Class'],axis=1)
#X=X.values
#print(X)
#print(y)

In [ ]:
#import collections as c
#counter = c.Counter(y)
#print(counter)

### **Data Preprocessing**

In [9]:
# Balance dataset using SMOTE
oversample = SMOTE(random_state=42)
X_resampled, y_resampled = oversample.fit_resample(X, y)

# Check class distribution after balancing
counter = c.Counter(y_resampled)
print("After SMOTE:", counter)

After SMOTE: Counter({0: 93000, 2: 93000, 1: 93000, 5: 93000, 8: 93000, 7: 93000, 4: 93000, 3: 93000, 9: 93000, 6: 93000})


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

### **Hybrid combination of LR-Bi-LSTM for 10 class classifications**

In [11]:
# Train logistic regression model
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print("Logistic Regression Accuracy:", lr_acc)
print(classification_report(y_test, lr_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Accuracy: 0.5269040457941205
              precision    recall  f1-score   support

           0       0.58      0.85      0.69     18390
           1       0.00      0.00      0.00       555
           2       0.00      0.00      0.00       468
           3       0.00      0.00      0.00      3403
           4       0.00      0.00      0.00      9040
           5       0.00      0.00      0.00      4949
           6       0.47      0.98      0.63     11630
           7       0.00      0.00      0.00      2749
           8       0.00      0.00      0.00       317
           9       0.00      0.00      0.00        34

    accuracy                           0.53     51535
   macro avg       0.10      0.18      0.13     51535
weighted avg       0.31      0.53      0.39     51535



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

In [13]:
# Prepare data for Bi-LSTM
num_classes = len(np.unique(y))
input_dim = X_train.shape[1]
y_train_onehot = to_categorical(y_train, num_classes=num_classes)
y_test_onehot = to_categorical(y_test, num_classes=num_classes)

X_train_lstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Define the Bi-LSTM model
lstm_model = Sequential([
    Bidirectional(LSTM(64, input_shape=(1, input_dim), activation='relu', return_sequences=True)),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])
lstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

#Define FGSM adversarial example generation
def generate_fgsm_adversarial_examples(model, x, y_true, epsilon=0.01):
    x_tensor = tf.convert_to_tensor(x)
    with tf.GradientTape() as tape:
        tape.watch(x_tensor)
        predictions = model(x_tensor)
        loss = tf.keras.losses.categorical_crossentropy(y_true, predictions)
    gradients = tape.gradient(loss, x_tensor)
    signed_gradients = tf.sign(gradients)
    adversarial_examples = x + epsilon * signed_gradients.numpy()
    return np.clip(adversarial_examples, 0, 1)

# Define PGD adversarial example generation
def generate_pgd_adversarial_examples(model, x, y_true, epsilon=0.01, alpha=0.01, iterations=10):
    x_adv = np.copy(x)
    for _ in range(iterations):
        fgsm_adv = generate_fgsm_adversarial_examples(model, x_adv, y_true, epsilon)
        x_adv += alpha * np.sign(fgsm_adv - x_adv)
        x_adv = np.clip(x_adv, 0, 1)
    return x_adv

# Generate adversarial examples for training
lstm_model.fit(X_train_lstm, y_train_onehot, epochs=15, batch_size=32, verbose=1)  # Pre-train the model
X_train_fgsm = generate_fgsm_adversarial_examples(lstm_model, X_train_lstm, y_train_onehot)
X_train_pgd = generate_pgd_adversarial_examples(lstm_model, X_train_lstm, y_train_onehot)

Epoch 1/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 49s 6ms/step - accuracy: 0.3361 - loss: 915293.0625
Epoch 2/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 74s 5ms/step - accuracy: 0.3619 - loss: 294.5859
Epoch 3/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 41s 5ms/step - accuracy: 0.3608 - loss: 123.1647
Epoch 4/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 34s 5ms/step - accuracy: 0.3608 - loss: 124.9984
Epoch 5/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 43s 6ms/step - accuracy: 0.3612 - loss: 52.7069
Epoch 6/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 37s 5ms/step - accuracy: 0.3629 - loss: 57.8570
Epoch 7/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 35s 5ms/step - accuracy: 0.3623 - loss: 6.8867
Epoch 8/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 39s 5ms/step - accuracy: 0.3606 - loss: 1.6866
Epoch 9/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 38s 6ms/step - accuracy: 0.3634 - loss: 2.5141
Epoch 10/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 38s 5ms/step - accuracy: 0.3617 - loss: 1.9076
Epoch 11/15
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 41s 5ms/step - accuracy: 0.3605 - loss: 25.1140
Epoch

In [14]:
# Combine original and adversarial data
X_train_combined = np.concatenate((X_train_lstm, X_train_fgsm, X_train_pgd), axis=0)
y_train_combined = np.concatenate((y_train_onehot, y_train_onehot, y_train_onehot), axis=0)

# Train Bi-LSTM on combined data
lstm_model.fit(X_train_combined, y_train_combined, epochs=15, batch_size=32, verbose=1)

# Evaluate Bi-LSTM
lstm_pred = lstm_model.predict(X_test_lstm)
lstm_pred_classes = np.argmax(lstm_pred, axis=1)
lstm_acc = accuracy_score(y_test, lstm_pred_classes)
print("Bi-LSTM Accuracy with Adversarial Training:", lstm_acc)
print(classification_report(y_test, lstm_pred_classes))

Epoch 1/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 109s 6ms/step - accuracy: 0.3615 - loss: 1.6874
Epoch 2/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 100s 5ms/step - accuracy: 0.3615 - loss: 1.6896
Epoch 3/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 143s 5ms/step - accuracy: 0.3631 - loss: 1.7164
Epoch 4/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 145s 5ms/step - accuracy: 0.3614 - loss: 1.6885
Epoch 5/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 140s 5ms/step - accuracy: 0.3629 - loss: 1.6863
Epoch 6/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 142s 5ms/step - accuracy: 0.3604 - loss: 1.7205
Epoch 7/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 146s 5ms/step - accuracy: 0.3619 - loss: 1.6906
Epoch 8/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 142s 5ms/step - accuracy: 0.3625 - loss: 1.7899
Epoch 9/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 141s 5ms/step - accuracy: 0.3622 - loss: 1.9016
Epoch 10/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 144s 6ms/step - accuracy: 0.3619 - loss: 2.3338
Epoch 11/15
19326/19326 ━━━━━━━━━━━━━━━━━━━━ 106s 5ms/step - accuracy: 0.3612 -

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
# Combine predictions from LR and Bi-LSTM
lr_probs = lr_model.predict_proba(X_test)
lstm_probs = lstm_model.predict(X_test_lstm)
combined_probs = (lr_probs + lstm_probs) / 2
combined_pred = np.argmax(combined_probs, axis=1)
combined_acc = accuracy_score(y_test, combined_pred)
print("Combined Model Accuracy:", combined_acc)
print(classification_report(y_test, combined_pred))

1611/1611 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Combined Model Accuracy: 0.3583778014941302
              precision    recall  f1-score   support

           0       0.36      0.98      0.53     18390
           1       0.00      0.00      0.00       555
           2       0.00      0.00      0.00       468
           3       0.00      0.00      0.00      3403
           4       0.00      0.00      0.00      9040
           5       0.00      0.00      0.00      4949
           6       0.28      0.04      0.07     11630
           7       0.00      0.00      0.00      2749
           8       0.00      0.00      0.00       317
           9       0.00      0.00      0.00        34

    accuracy                           0.36     51535
   macro avg       0.06      0.10      0.06     51535
weighted avg       0.19      0.36      0.20     51535



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns

# Calculate metrics
y_true = np.argmax(y_test, axis=1)  # True labels
y_pred = test_bi_lstm_predictions_classes  # Predicted labels

# Accuracy, Precision, Recall, F1-Score (weighted and per class)
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average=None)
recall = recall_score(y_true, y_pred, average=None)
f1 = f1_score(y_true, y_pred, average=None)

# Print metrics for each class
print(f"Accuracy: {accuracy:.20f}")
for i in range(len(precision)):
    print(f"Class {i}:")
    print(f"  Precision: {precision[i]:.20f}")
    print(f"  Recall:    {recall[i]:.20f}")
    print(f"  F1-Score:  {f1[i]:.20f}")

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
# Classification report
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, digits=8))

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Display confusion matrix as text
print("\nConfusion Matrix:\n")
print(conf_matrix)

In [ ]:
# Plot confusion matrix as heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(conf_matrix.shape[0]), yticklabels=range(conf_matrix.shape[0]))
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix Heatmap')
plt.show()